In [0]:
# p mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data_224.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Mounted at /content/gdrive
replace resized/9733.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
ls: cannot access 'drive': Transport endpoint is not connected
drive/  gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [0]:
 # load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""
    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation=augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"])/255 for _, item in items.iterrows()])
        if self.augmentation is not None:
          image = self.augmentation.flow(image, shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
       # print("A")
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
        #    print("B")
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state=1)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
58,resized/59.jpg,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
2106,resized/2107.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0
5206,resized/5207.jpg,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0
1163,resized/1164.jpg,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
13534,resized/13536.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1


In [0]:
## create train and validation data generators
import skimage.exposure
def AHE(img):
    img_adapteq = skimage.exposure.equalize_adapthist(img, clip_limit=0.03)
    return img_adapteq
train_gen = PersonDataGenerator(train_df, batch_size=32, augmentation=ImageDataGenerator(horizontal_flip=True, vertical_flip=True, preprocessing_function=AHE))
valid_gen = PersonDataGenerator(val_df, batch_size=32, shuffle=False, augmentation=ImageDataGenerator(preprocessing_function=AHE))

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float64 to float32
  .format(dtypeobj_in, dtypeobj_out))


{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Convolution2D, Flatten, MaxPooling2D, AveragePooling2D, Activation, BatchNormalization
from keras.models import Model
from keras.layers import Input

def convx(input_img, ch, ker1, ker2, type):
  x = Conv2D(ch, (ker1,ker2), padding=type, use_bias=False)(input_img)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  return x 

input_img = Input(shape=(224,224,3))
x = convx(input_img, 32, 3, 3, 'same')
x = convx(x, 32, 3, 3, 'same')
x = MaxPooling2D((2,2), strides=2)(x)
x = Dropout(0.1)(x)

x = convx(x, 64, 3, 3, 'same')
x = convx(x, 64, 3, 3, 'same')
x = MaxPooling2D((2,2), strides=2)(x)
x = Dropout(0.15)(x)

x = convx(x, 128, 3, 3, 'same')
x = convx(x, 128, 3, 3, 'same')
x = MaxPooling2D((2,2), strides=2)(x)
x = Dropout(0.2)(x)

x = convx(x, 256, 3, 3, 'same')
x = convx(x, 256, 3, 3, 'same')
x = MaxPooling2D((2,2), strides=2)(x)
x = Dropout(0.25)(x)

x = convx(x, 512, 3, 3, 'same')
x = convx(x, 512, 3, 3, 'same')
x = MaxPooling2D((2,2), strides=2)(x)
x = Dropout(0.25)(x)

x = AveragePooling2D(7)(x)

model2 = Model(inputs=input_img, outputs=x)

model2.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 224, 224, 32)      864       
_________________________________________________________________
batch_normalization_11 (Batc (None, 224, 224, 32)      128       
_________________________________________________________________
activation_11 (Activation)   (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 224, 224, 32)      9216      
_________________________________________________________________
batch_normalization_12 (Batc (None, 224, 224, 32)      128       
_________________________________________________________________
activation_12 (Activation)   (None, 224, 224, 32)      0   

In [0]:
#backbone = VGG16(
#    weights="imagenet", 
#    include_top=False, 
#    input_tensor=Input(shape=(224, 224, 3))
#)
backbone = model2

neck = backbone.output
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)


def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output", kernel_initializer='glorot_uniform'
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))

model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

In [0]:
# freeze backbone
for layer in backbone.layers:
	layer.trainable = True

In [0]:
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
 	"age_output": "mse",
 	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy"
 }
loss_weights = {"gender_output": 1.5, "image_quality_output": 1.5, "age_output": 2, "weight_output": 1.2, "bag_output": 1.2, "footwear_output": 1.5, "pose_output": 1.0, "emotion_output": 1.0}

In [0]:
from keras.callbacks import *
filepath="/content/gdrive/My Drive/MyCNN/epochs:_1.hdf5"
def scheduler(epoch, lr):
  print('Learning rate: ', lr)
  return lr
lr_scheduler = LearningRateScheduler(scheduler)
lr_reducer = ReduceLROnPlateau(monitor='loss',
                               factor=0.5,
                               cooldown=0,
                               patience=15,
                               min_lr=0.5e-6)
csv_logger = CSVLogger('/content/gdrive/My Drive/MyCNN/_log.csv', append=True, separator=',')
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [lr_scheduler, lr_reducer]

opt = SGD(lr=0.1, momentum=0.9, nesterov=True, decay=0.1)
with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model.compile(
        optimizer=opt,
        loss= losses, 
        loss_weights=loss_weights, 
        metrics=["accuracy"]
        )
        model_info = model.fit_generator(
        generator=train_gen,
        validation_data=valid_gen,
        use_multiprocessing=True,
        callbacks=callbacks_list,
        initial_epoch=0,
        workers=6, 
        epochs=300,
        verbose=1 
        )

#model.load_weights('/content/gdrive/My Drive/MyCNN/epochs: 1.hdf5')

Epoch 1/300
Learning rate:  0.10000000149011612
360/360 [==============================] - 273s 758ms/step - loss: 10.4454 - gender_output_loss: 0.7676 - image_quality_output_loss: 1.2424 - age_output_loss: 0.1485 - weight_output_loss: 1.3090 - bag_output_loss: 1.2878 - footwear_output_loss: 1.3108 - pose_output_loss: 1.0220 - emotion_output_loss: 1.0289 - gender_output_acc: 0.5556 - image_quality_output_acc: 0.5440 - age_output_acc: 0.4023 - weight_output_acc: 0.6193 - bag_output_acc: 0.5520 - footwear_output_acc: 0.5077 - pose_output_acc: 0.6129 - emotion_output_acc: 0.7117 - val_loss: 8.4925 - val_gender_output_loss: 0.6826 - val_image_quality_output_loss: 0.9818 - val_age_output_loss: 0.1487 - val_weight_output_loss: 0.9811 - val_bag_output_loss: 0.9407 - val_footwear_output_loss: 0.9923 - val_pose_output_loss: 0.9262 - val_emotion_output_loss: 0.9776 - val_gender_output_acc: 0.5645 - val_image_quality_output_acc: 0.5551 - val_age_output_acc: 0.3656 - val_weight_output_acc: 0.6394 